In [1]:
# muzero in a notebook on cartpole

In [5]:
%pylab inline
import tensorflow as tf
#import tensorflow.keras.backend as K
import numpy as np
import gym
from tqdm import trange
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.layers import *
np.set_printoptions(suppress=True)

def toh(x,n):
    ret = np.zeros([n])
    ret[x] = 1.0
    return ret
def bstack(bb):
    ret = [[x] for x in bb[0]]
    for i in range(1, len(bb)):
        for j in range(len(bb[i])):
            ret[j].append(bb[i][j])
    return [np.array(x) for x in ret]

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.7/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['concatenate', 'add', 'maximum', 'subtract', 'average', 'dot', 'multiply', 'f', 'minimum']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [6]:
env = gym.make("CartPole-v0")
env.observation_space, env.action_space

(Box(4,), Discrete(2))

In [7]:
S_DIM = 4

# h: representation function
# s_0 = h(o_1...o_t)
x = o_0 = Input(env.observation_space.shape)
x = Dense(S_DIM)(x)
s_0 = Activation('tanh')(x)
unh = Model(o_0, s_0, name="h")
def ht(o_0):
    return h.predict(o_0[None])[0]

# g: dynamics function (recurrent in state?) old_state+action -> state+reward
# r_k, s_k = g(s_k-1, a_k)
s_km1 = Input(S_DIM)
a_k = Input(env.action_space.n)
x = Concatenate()([s_km1, a_k])
x = Dense(64)(x)
x = Activation('elu')(x)
x = Dense(64)(x)
x = Activation('elu')(x)
s_k = Dense(S_DIM, name='s_k')(x)
r_k = Dense(1, name='r_k')(x)
g = Model([s_km1, a_k], [r_k, s_k], name="g")
g.compile('adam', 'mse')
def gt(s_km1, a_k):
    r_k, s_k = g.predict([s_km1[None], a_k[None]])
    return r_k[0], s_k[0]

# f: prediction function -- state -> policy+value
# p_k, v_k = f(s_k)
x = s_k = Input(S_DIM)
x = Dense(32)(x)
x = Activation('tanh')(x)
p_k = Dense(env.action_space.n)(x)
p_k = Activation('softmax', name='p_k')(p_k)
v_k = Dense(1, name='v_k')(x)
f = Model(s_k, [p_k, v_k], name="f")
f.compile('adam', 'mse')
def ft(s_k):
    p_k, v_k = f.predict(s_k[None])
    return p_k[0], v_k[0]

In [8]:
# it's using the dynamics function for rollout search
K = 5
gamma = 0.95

# represent
o_0 = Input(env.observation_space.shape, name="o_0")
# don't use the h function for now
#s_km1 = h(o_0)
s_km1 = o_0

# rollout with dynamics
# p_k, v_k, r_k = mu(o_0, a_1_k)
a_all, mu_all = [], []
for k in range(K):
    a_k = Input(env.action_space.n, name="a_%d" % k)
    r_k, s_k  = g([s_km1, a_k])
    
    # predict
    p_k, v_k = f([s_k])
    
    # store
    a_all.append(a_k)
    mu_all.append([p_k, v_k, r_k])
    s_km1 = s_k

# put in the first observation and actions
#   need policy from search
#   need values from sum of rewards + last state value (real state?)
#   need rewards
#a_all = Concatenate()(a_all)
mu = Model([o_0, a_all], mu_all)
mu.compile('adam', 'mse')
mu.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
o_0 (InputLayer)                [(None, 4)]          0                                            
__________________________________________________________________________________________________
a_0 (InputLayer)                [(None, 2)]          0                                            
__________________________________________________________________________________________________
g (Model)                       [(None, 1), (None, 4 4933        o_0[0][0]                        
                                                                 a_0[0][0]                        
                                                                 g[1][1]                          
                                                                 a_1[0][0]                    

In [17]:
import itertools

aopts = list(itertools.product([0,1], repeat=K))
aoptss = np.array([[toh(x, 2) for x in aa] for aa in aopts])
aoptss = aoptss.swapaxes(0,1)
aoptss = [aoptss[x] for x in range(5)]

def search(o_0):
    # 2^5 rollout isn't too bad, explore them all
    # maximize value
    o_0s = np.repeat(np.array(o_0)[None], len(aopts), axis=0)
    ret = mu.predict([o_0s]+aoptss)
    v = [(ret[-2][i][0], aopts[i]) for i in range(len(ret[-2]))]
    return sorted(v, reverse=True)

env.reset()
search(env.state)

[(13.801957, (1, 0, 1, 0, 1)),
 (13.794019, (0, 1, 1, 1, 0)),
 (13.793693, (0, 1, 1, 0, 1)),
 (13.792161, (1, 0, 0, 1, 1)),
 (13.766726, (1, 0, 1, 1, 0)),
 (13.763342, (1, 1, 0, 0, 1)),
 (13.755687, (0, 1, 0, 1, 1)),
 (13.705733, (0, 0, 1, 1, 1)),
 (13.683206, (1, 1, 0, 1, 0)),
 (13.522788, (1, 1, 1, 0, 0)),
 (12.379579, (0, 1, 1, 1, 1)),
 (12.347724, (1, 1, 0, 0, 0)),
 (12.130088, (1, 0, 1, 1, 1)),
 (12.074832, (1, 0, 1, 0, 0)),
 (11.83241, (1, 0, 0, 1, 0)),
 (11.82828, (0, 1, 1, 0, 0)),
 (11.777593, (1, 1, 0, 1, 1)),
 (11.623328, (1, 0, 0, 0, 1)),
 (11.567865, (0, 1, 0, 1, 0)),
 (11.367116, (0, 0, 1, 1, 0)),
 (11.343383, (0, 1, 0, 0, 1)),
 (11.328216, (1, 1, 1, 0, 1)),
 (11.130788, (0, 0, 1, 0, 1)),
 (10.976069, (0, 0, 0, 1, 1)),
 (10.821125, (1, 1, 1, 1, 0)),
 (8.384873, (1, 0, 0, 0, 0)),
 (8.063352, (0, 1, 0, 0, 0)),
 (8.006327, (1, 1, 1, 1, 1)),
 (7.8176646, (0, 0, 1, 0, 0)),
 (7.637002, (0, 0, 0, 1, 0)),
 (7.4666595, (0, 0, 0, 0, 1)),
 (5.021906, (0, 0, 0, 0, 0))]

In [9]:
env.reset()
for _ in range(10):
    x,y = [],[]
    for _ in range(16):
        o_0 = np.copy(env.state)
        acts = search(o_0)
        v_k, a_0k = acts[0]
        #v_k, a_0k = acts[random.randint(0, len(acts))]
        a_0koh = [toh(x, 2) for x in a_0k]
        x.append([o_0]+a_0koh)

        # actually act with best value policy
        rs = []
        for i in range(K):
            _, r, done, _ = env.step(a_0k[i])
            if done:
                #r = -50
                env.reset()
            rs.append(r)

        # compute values
        v_0k = [v_k]
        for r in rs[::-1]:
            v_0k = [v_0k[0]*gamma + r] + v_0k

        yl = []
        for i in range(K):
            yl += [a_0koh[i], v_0k[i], rs[i]]
        y.append(yl)
            
    ll = mu.fit(bstack(x), bstack(y), verbose=0)
    loss = ll.history['loss']
    print(loss)

[38.75679397583008]
[37.6257438659668]
[36.63232421875]
[36.23903274536133]
[34.34711837768555]
[33.506534576416016]
[32.88059616088867]
[31.765249252319336]
[31.22316551208496]
[30.173458099365234]


In [16]:
# can act?
# not yet
env.reset()
for sn in range(100):
    ret = search(env.state)
    v,aa = ret[0]
    print(aa[0], v)
    env.render()
    _,r,done,_ = env.step(aa[0])
    if done:
        print("DONE", sn)
        break

1 13.733291
0 13.684348
1 13.727105
0 13.70251
1 13.719481
0 13.708284
1 13.713237
0 13.70372
1 13.708823
0 13.688849
1 13.704804
0 13.661925
1 13.702993
0 13.619447
1 13.707948
0 13.555988
1 13.708226
1 13.499552
0 13.69616
1 13.516931
0 13.570954
1 13.54375
1 13.297528
0 13.194484
0 13.342956
0 13.355192
0 13.3951235
0 13.186452
0 13.456133
1 13.270425
0 13.476573
1 13.379913
0 13.480659
1 13.4441185
0 13.501527
1 13.4934435
0 13.550317
1 13.534984
0 13.614503
1 13.54923
0 13.673642
1 13.488374
1 13.7078085
1 13.588584
0 13.720469
1 13.552429
0 13.744243
0 13.60601
1 13.756001
0 13.706918
1 13.765517
0 13.787965
1 13.769981
0 13.83617
1 13.727666
0 13.869744
0 13.643475
1 13.881594
0 13.809009
1 13.742021
1 13.878515
1 13.830179
0 13.92369
0 13.75325
1 13.856716
1 13.857788
1 13.91124
0 13.942697
1 13.822467
0 13.982777
0 13.82396
1 13.892095
1 13.896438
1 13.955777
0 13.980973
1 13.925969
0 14.022838
1 13.89408
0 14.048903
1 13.861461
0 14.057389
1 13.846858
0 14.056499
1 13.851026


In [10]:
# Test: Can the f function learn a policy+value model

x_s = []
y_p = []
y_v = []

for _ in trange(1000):
    env.reset()
    rs = []
    for _ in range(100):
        a_k = env.action_space.sample()
        _, r, done, _ = env.step(a_k)
        x_s.append(env.state)
        y_p.append(toh(a_k, 2))
        rs.append(r)
    v_0k = [0]
    for r in rs[::-1]:
        v_0k = [v_0k[0]*gamma + r] + v_0k
    y_v += v_0k[:-1]
x_s = np.array(x_s)
y_p, y_v = np.array(y_p), np.array(y_v)
    
f.fit(x_s, [y_p, y_v], batch_size=32, epochs=5, shuffle=1)

  0%|          | 0/1000 [00:00<?, ?it/s]/usr/local/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
100%|██████████| 1000/1000 [00:02<00:00, 491.26it/s]


Train on 100000 samples
Epoch 1/5
100000/100000 [==============================] - 3s 30us/sample - loss: 6.4070 - p_k_loss: 0.2418 - v_k_loss: 6.1652
Epoch 2/5
100000/100000 [==============================] - 3s 28us/sample - loss: 2.7266 - p_k_loss: 0.2327 - v_k_loss: 2.4938
Epoch 3/5
100000/100000 [==============================] - 3s 28us/sample - loss: 2.3641 - p_k_loss: 0.2297 - v_k_loss: 2.1344
Epoch 4/5
100000/100000 [==============================] - 3s 28us/sample - loss: 2.2851 - p_k_loss: 0.2287 - v_k_loss: 2.0563
Epoch 5/5
100000/100000 [==============================] - 3s 27us/sample - loss: 2.2572 - p_k_loss: 0.2283 - v_k_loss: 2.0290


In [11]:
# Test: Can the g function learn a dynamics model
# Answer: yes!
# Sadly, with h, this is hard to test in the whole system

x_s, x_a = [], []
y_r, y_s = [], []

for _ in trange(1000):
    env.reset()
    for _ in range(100):
        #env.render()
        a_k = env.action_space.sample()
        x_s.append(env.state)
        x_a.append(toh(a_k, env.action_space.n))
        _, r, done, _ = env.step(a_k)
        y_r.append([r])
        y_s.append(env.state)
        #if done:
        #    break
    env.close()
x_s, x_a = np.array(x_s), np.array(x_a)
y_r, y_s = np.array(y_r), np.array(y_s)

g.fit([x_s, x_a], [y_r, y_s], batch_size=32, epochs=5, shuffle=1)

100%|██████████| 1000/1000 [00:02<00:00, 454.19it/s]


Train on 100000 samples
Epoch 1/5
100000/100000 [==============================] - 4s 35us/sample - loss: 0.1094 - r_k_loss: 0.0491 - s_k_loss: 0.0603
Epoch 2/5
100000/100000 [==============================] - 3s 33us/sample - loss: 0.0285 - r_k_loss: 0.0211 - s_k_loss: 0.0073
Epoch 3/5
100000/100000 [==============================] - 3s 33us/sample - loss: 0.0223 - r_k_loss: 0.0166 - s_k_loss: 0.0057
Epoch 4/5
100000/100000 [==============================] - 3s 33us/sample - loss: 0.0201 - r_k_loss: 0.0152 - s_k_loss: 0.0049
Epoch 5/5
100000/100000 [==============================] - 3s 33us/sample - loss: 0.0186 - r_k_loss: 0.0144 - s_k_loss: 0.0042


In [13]:
# test the dynamics model
env.reset()
for _ in range(100):
    env.render()
    a_k = env.action_space.sample()
    r_k, s_k = gt(env.state, toh(a_k, env.action_space.n))
    p_k, v_k = ft(s_k)
    print(a_k, r_k, v_k, np.argmax(p_k))
    env.unwrapped.state = s_k
    #env.step(a_k)
env.close()

1 [1.0660192] [13.723018] 1
1 [1.0719117] [13.409712] 1
0 [1.0903379] [13.763711] 1
0 [1.0704638] [12.975025] 1
0 [1.0384701] [10.96083] 0
1 [0.9717561] [11.8278] 1
0 [0.9688573] [9.245917] 0
0 [0.8637582] [6.5032196] 0
0 [0.69667184] [4.113181] 0
1 [0.45217487] [3.8137937] 0
0 [0.3660025] [1.9988377] 0
1 [0.1688199] [1.580481] 0
0 [0.11066136] [0.6227689] 0
1 [-0.00078909] [0.32047033] 1
1 [-0.03729492] [0.09122348] 1
0 [-0.03628473] [-0.05037463] 1
0 [-0.04080047] [-0.06838822] 1
1 [-0.03352335] [-0.07491505] 1
0 [-0.00827189] [-0.04505146] 1
1 [-0.00728083] [-0.02573025] 1
0 [0.02716774] [-0.01078022] 1
1 [-0.00327633] [0.00095594] 1
1 [-0.01499498] [0.00741518] 1
1 [-0.03762173] [0.01040471] 1
0 [-0.04375656] [0.01101959] 1
1 [-0.08673285] [0.01187098] 1
1 [-0.11904402] [0.01234138] 1
0 [-0.13987117] [0.01221573] 1
0 [-0.15327391] [0.01204026] 1
0 [-0.1638348] [0.01181543] 1
1 [-0.17894322] [0.01218116] 1
1 [-0.19830191] [0.01247466] 1
0 [-0.21190256] [0.0121938] 1
1 [-0.21048354] 